In [99]:
# Imports
import pandas as pd

In [100]:
path = 'https://raw.githubusercontent.com/medamer/B2P-data/main/Combined_dataset.csv'
df = pd.read_csv(path)

In [101]:
df.head()

,Bridge Name,Bridge Opportunity: Project Code,Bridge Opportunity: Needs Assessment,Bridge Opportunity: Level 1 Government,Bridge Opportunity: Level 2 Government,Bridge Opportunity: GPS (Latitude),Bridge Opportunity: GPS (Longitude),Bridge Opportunity: Bridge Type,Bridge Opportunity: Span (m),Bridge Opportunity: Individuals Directly Served,Bridge Opportunity: Comments,Current crossing method,Nearest all-weather crossing point,Days per year river is flooded,Flood duration during rainy season,Market access blocked by river,Education access blocked by river,Health access blocked by river,Other access blocked by river,Primary occupations,Primary crops grown,River crossing deaths in last 3 years,River crossing injuries in last 3 years,Cell service quality,4WD Accessibility,Name of nearest city,Name of nearest paved or sealed road,Bridge classification,Flag for Rejection,Rejection Reason,Bridge Type,Estimated span (m),Height differential between banks,Bridge Opportunity: CaseSafeID,Country,B2P Fiscal Year,Project Type,Good Site
0,Bukinga,1009317,Rwanda Needs Assessment 2018,Southern Province,Nyaruguru,-2.760833,29.488056,Suspended Bridge,NaN,NaN,Steep slopes to deal with for anchors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Huye,NaN,NaN,Yes,There is a planned vehicular road with a vehic...,NaN,NaN,NaN,006f100000asFErAAM,NaN,NaN,NaN,0.0
1,Kagarama,1007325,NaN,Western Province,Nyamasheke,-2.322534,29.141944,Suspension Bridge,48.0,4000.0,NaN,Timber Bridge,2 km,120.0,12hours,"Schools, hospital, markets and government serv...",NaN,NaN,NaN,farming,banana,2.0,15.0,Fair,Seasonally,Kibogora-Kabuga,Karongi-Nyamasheke,NaN,NaN,NaN,NaN,10.0,0.7,006f100000a82QzAAI,NaN,NaN,NaN,1.0
2,Karambi,1007374,NaN,Southern Province,Huye,-2.513333,29.612778,Suspension Bridge,50.0,12500.0,Likely piloting steel decking on this bridge. ...,Timber Bridge,5 km,60.0,1day,"Markets,farms blocked by this river both sides",Education blocked by this river both sides,Health services blocked by this river both sides,"Potable water, places of worship ,places for r...",Farming and small business,"banana, bean, corn",3.0,15.0,Good,Never,Nyamagabe,Huye-Nyamagabe,Standard,NaN,NaN,Suspension,45.0,1.0,006f100000a86DaAAI,NaN,NaN,NaN,1.0
3,Rugeti,1007466,Rwanda Needs Assessment 2018,Southern Province,Nyamagabe,-2.476407,29.420800,Suspension Bridge,68.0,4363.0,"3000-6000 people directly served, Elevation:21...",Timber,None,60.0,All time in April and May and it is rarely to ...,Markets,Primary and Secondary schools,Health Center,Churches and Government offices,"Farming of large tea plantation, wheat farms","bean, cow pea, sweet potato, tea, wheat",0.0,2.0,Fair,Seasonally,Huye,Nyamagabe-Rusizi,Standard,No,NaN,Suspended,47.4,1.1,006f100000a86F4AAI,NaN,NaN,NaN,1.0
4,Nyakabuye - Nkomane,1007495,Rwanda Needs Assessment 2018,Southern Province,Nyamagabe,-2.233022,29.575130,Suspension Bridge,78.0,2790.0,"Not Duplicate 500-1000 people directly served,...",Timber Bridge,5 kilometers,35.0,"April To May, September to December, 12 hours ...","Birambo market, Gasura market, kirinda market(...","Groupe scholar Gasave( 1,2)","Kirinda Hospital(villages 1,2)",None,Farming,"bean, cassava, maiz, sorghum, sweet potato",4.0,4.0,Good,Never,Nyanza,Nyamagabe,Standard,No,NaN,Suspended,90.0,1.1,006f100000a86FXAAY,NaN,NaN,NaN,1.0


In [102]:
df.rename(columns={'Bridge Name': 'bridge_name',
                   'Bridge Opportunity: Project Code': 'bridge_opportunity_project_code',
                   'Bridge Opportunity: Needs Assessment': 'bridge_opportunity_needs_assessment',
                   'Bridge Opportunity: Level 1 Government': 'bridge_opportunity_level1_government',
                   'Bridge Opportunity: Level 2 Government': 'bridge_opportunity_level2_government',
                   'Bridge Opportunity: GPS (Latitude)': 'bridge_opportunity_gps_latitude',
                   'Bridge Opportunity: GPS (Longitude)': 'bridge_opportunity_gps_longitude',
                   'Bridge Opportunity: Bridge Type': 'bridge_opportunity_bridge_type',
                   'Bridge Opportunity: Span (m)': 'bridge_opportunity_span_m',
                   'Bridge Opportunity: Individuals Directly Served': 'bridge_opportunity_individuals_directly_served',
                   'Bridge Opportunity: Comments': 'bridge_opportunity_comments',
                   'Current crossing method': 'current_crossing_method',
                   'Nearest all-weather crossing point': 'nearest_all_weather_crossing_point',
                   'Days per year river is flooded': 'days_per_year_river_is_flooded',
                   'Flood duration during rainy season': 'flood_duration_during_rainy_season',
                   'Market access blocked by river': 'market_access_blocked_by_river',
                   'Education access blocked by river': 'education_access_blocked_by_river',
                   'Health access blocked by river': 'health_access_blocked_by_river',
                   'Other access blocked by river': 'other_access_blocked_by_river', 'Primary occupations': 'primary_occupations',
                   'Primary crops grown': 'primary_crops_grown',
                   'River crossing deaths in last 3 years': 'river_crossing_deaths_in_last_3_years',
                   'River crossing injuries in last 3 years': 'river_crossing_injuries_in_last_3_years',
                   'Cell service quality': 'incident_descriptions',
                   '4WD Accessibility': 'four_wd _accessibility', 'Name of nearest city': 'name_of_nearest_city',
                   'Name of nearest paved or sealed road': 'name_of_nearest_paved_or_sealed_road',
                   'Bridge classification': 'bridge_classification', 'Flag for Rejection': 'flag_for_rejection',
                   'Rejection Reason': 'rejection_reason', 'Bridge Type': 'bridge_type',
                   'Estimated span (m)': 'estimated_span_m', 'Height differential between banks': 'height_differential_between_banks',
                   'Bridge Opportunity: CaseSafeID': 'bridge_opportunity_casesafeid', 'Country': 'country',
                   'B2P Fiscal Year': 'b2p_fiscal_year',
                   'Project Type': 'project_type', 'Good Site': 'good_site'}, inplace=True)
df.columns

Index(['bridge_name', 'bridge_opportunity_project_code',
       'bridge_opportunity_needs_assessment',
       'bridge_opportunity_level1_government',
       'bridge_opportunity_level2_government',
       'bridge_opportunity_gps_latitude', 'bridge_opportunity_gps_longitude',
       'bridge_opportunity_bridge_type', 'bridge_opportunity_span_m',
       'bridge_opportunity_individuals_directly_served',
       'bridge_opportunity_comments', 'current_crossing_method',
       'nearest_all_weather_crossing_point', 'days_per_year_river_is_flooded',
       'flood_duration_during_rainy_season', 'market_access_blocked_by_river',
       'education_access_blocked_by_river', 'health_access_blocked_by_river',
       'other_access_blocked_by_river', 'primary_occupations',
       'primary_crops_grown', 'river_crossing_deaths_in_last_3_years',
       'river_crossing_injuries_in_last_3_years', 'incident_descriptions',
       'four_wd _accessibility', 'name_of_nearest_city',
       'name_of_nearest_paved_o

In [103]:
# Split the dataset:
# Positives:
# positive = (
#     (df['senior_engineering_review_conducted']=='Yes') & 
#     (df['bridge_opportunity_stage'].isin(['Complete', 'Prospecting', 'Confirmed', 'Under Construction']))
# )

# # Negatives:
# negative = (
#     (df['senior_engineering_review_conducted']=='Yes') & 
#     (df['bridge_opportunity_stage'].isin(['Rejected', 'Cancelled']))
# )

# # Unknown:
# unknown = df['senior_engineering_review_conducted'].isna()

In [104]:
# Create a new column named "Good Site." This is the target to predict.
# Assign a 1 for the positive class, 0 for the negative class and -1 for unkown class.
# df.loc[positive, 'good_site'] = 1
# df.loc[negative, 'good_site'] = 0
# df.loc[unknown, 'good_site'] = -1

In [105]:
# Because these columns were used to derive the target, 
# We can't use them as features, or it would be leakage.
# df = df.drop(columns=['senior_engineering_review_conducted', 'bridge_opportunity_stage'])
df['good_site'].value_counts()

-1.0    1383
 1.0     404
 0.0      24
Name: good_site, dtype: int64

In [106]:
df.head()

,bridge_name,bridge_opportunity_project_code,bridge_opportunity_needs_assessment,bridge_opportunity_level1_government,bridge_opportunity_level2_government,bridge_opportunity_gps_latitude,bridge_opportunity_gps_longitude,bridge_opportunity_bridge_type,bridge_opportunity_span_m,bridge_opportunity_individuals_directly_served,bridge_opportunity_comments,current_crossing_method,nearest_all_weather_crossing_point,days_per_year_river_is_flooded,flood_duration_during_rainy_season,market_access_blocked_by_river,education_access_blocked_by_river,health_access_blocked_by_river,other_access_blocked_by_river,primary_occupations,primary_crops_grown,river_crossing_deaths_in_last_3_years,river_crossing_injuries_in_last_3_years,incident_descriptions,four_wd _accessibility,name_of_nearest_city,name_of_nearest_paved_or_sealed_road,bridge_classification,flag_for_rejection,rejection_reason,bridge_type,estimated_span_m,height_differential_between_banks,bridge_opportunity_casesafeid,country,b2p_fiscal_year,project_type,good_site
0,Bukinga,1009317,Rwanda Needs Assessment 2018,Southern Province,Nyaruguru,-2.760833,29.488056,Suspended Bridge,NaN,NaN,Steep slopes to deal with for anchors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Huye,NaN,NaN,Yes,There is a planned vehicular road with a vehic...,NaN,NaN,NaN,006f100000asFErAAM,NaN,NaN,NaN,0.0
1,Kagarama,1007325,NaN,Western Province,Nyamasheke,-2.322534,29.141944,Suspension Bridge,48.0,4000.0,NaN,Timber Bridge,2 km,120.0,12hours,"Schools, hospital, markets and government serv...",NaN,NaN,NaN,farming,banana,2.0,15.0,Fair,Seasonally,Kibogora-Kabuga,Karongi-Nyamasheke,NaN,NaN,NaN,NaN,10.0,0.7,006f100000a82QzAAI,NaN,NaN,NaN,1.0
2,Karambi,1007374,NaN,Southern Province,Huye,-2.513333,29.612778,Suspension Bridge,50.0,12500.0,Likely piloting steel decking on this bridge. ...,Timber Bridge,5 km,60.0,1day,"Markets,farms blocked by this river both sides",Education blocked by this river both sides,Health services blocked by this river both sides,"Potable water, places of worship ,places for r...",Farming and small business,"banana, bean, corn",3.0,15.0,Good,Never,Nyamagabe,Huye-Nyamagabe,Standard,NaN,NaN,Suspension,45.0,1.0,006f100000a86DaAAI,NaN,NaN,NaN,1.0
3,Rugeti,1007466,Rwanda Needs Assessment 2018,Southern Province,Nyamagabe,-2.476407,29.420800,Suspension Bridge,68.0,4363.0,"3000-6000 people directly served, Elevation:21...",Timber,None,60.0,All time in April and May and it is rarely to ...,Markets,Primary and Secondary schools,Health Center,Churches and Government offices,"Farming of large tea plantation, wheat farms","bean, cow pea, sweet potato, tea, wheat",0.0,2.0,Fair,Seasonally,Huye,Nyamagabe-Rusizi,Standard,No,NaN,Suspended,47.4,1.1,006f100000a86F4AAI,NaN,NaN,NaN,1.0
4,Nyakabuye - Nkomane,1007495,Rwanda Needs Assessment 2018,Southern Province,Nyamagabe,-2.233022,29.575130,Suspension Bridge,78.0,2790.0,"Not Duplicate 500-1000 people directly served,...",Timber Bridge,5 kilometers,35.0,"April To May, September to December, 12 hours ...","Birambo market, Gasura market, kirinda market(...","Groupe scholar Gasave( 1,2)","Kirinda Hospital(villages 1,2)",None,Farming,"bean, cassava, maiz, sorghum, sweet potato",4.0,4.0,Good,Never,Nyanza,Nyamagabe,Standard,No,NaN,Suspended,90.0,1.1,006f100000a86FXAAY,NaN,NaN,NaN,1.0


In [107]:
!pip install category_encoders

In [108]:
# Imports:
from collections import Counter
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report

In [109]:
# Select features:
features = [
       'bridge_opportunity_needs_assessment',
       'bridge_opportunity_gps_latitude', 'bridge_opportunity_gps_longitude',
       'bridge_opportunity_bridge_type', 'bridge_opportunity_span_m',
       'bridge_opportunity_individuals_directly_served','current_crossing_method',
       'nearest_all_weather_crossing_point', 'days_per_year_river_is_flooded',
       'flood_duration_during_rainy_season', 'market_access_blocked_by_river',
       'education_access_blocked_by_river', 'health_access_blocked_by_river',
       'other_access_blocked_by_river', 'primary_occupations',
       'primary_crops_grown', 'river_crossing_deaths_in_last_3_years',
       'river_crossing_injuries_in_last_3_years', 'incident_descriptions', 'bridge_classification',
       'flag_for_rejection', 'rejection_reason', 'bridge_type','height_differential_between_banks']

In [110]:
# Use the classifier model:
classifier = RandomForestClassifier

# Split the data to train and test sets:
data = df[(df['good_site']== 0) | (df['good_site']== 1)]
test = df[df['good_site']== -1]
data.shape, test.shape

((428, 38), (1383, 38))

In [111]:
# Split the data into train, validation, test and target:
# X_train = train[features]
# y_train = train['good_site']
train, val = train_test_split(data, test_size=0.2, random_state=42)
X_train = train[features]
y_train = train['good_site']
X_val = val[features]
y_val = val['good_site']

X_test = test[features]

X_train.shape, y_train.shape,X_val.shape, y_val.shape, X_test.shape

((342, 24), (342,), (86, 24), (86,), (1383, 24))

In [112]:
# Build a normal model:
pipeline = make_pipeline(ce.OrdinalEncoder(),
                        SimpleImputer(strategy = 'mean'),
                        classifier(random_state=42))
model = pipeline.fit(X_train, y_train)
prediction = model.predict(X_val)

In [113]:
# Model Summary:
prediction = pd.DataFrame(prediction, columns=['good_site'])
prediction.value_counts()

good_site
1.0          77
0.0           9
dtype: int64

In [114]:
# Build a model using oversampling technique SMOTE:
smote_pipeline = make_pipeline_imb(ce.OrdinalEncoder(),
                                   SimpleImputer(strategy = 'mean'),
                                   SMOTE(random_state=1),
                                   classifier(random_state=42))
smote_model = smote_pipeline.fit(X_train, y_train)
smote_prediction = smote_model.predict(X_val)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [115]:
smote_prediction = pd.DataFrame(smote_prediction, columns=['good_site'])
smote_prediction.value_counts()

good_site
1.0          76
0.0          10
dtype: int64

In [116]:
# Build a model using undersampling technique:
nm_pipeline = make_pipeline_imb(ce.OrdinalEncoder(),
                               SimpleImputer(strategy = 'mean'),
                               NearMiss(random_state=42),
                               classifier(random_state=42))
nm_model = nm_pipeline.fit(X_train, y_train)
nm_prediction = nm_model.predict(X_val)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [117]:
nm_prediction = pd.DataFrame(nm_prediction, columns=['good_site'])
nm_prediction.value_counts()

good_site
1.0          71
0.0          15
dtype: int64

In [118]:
# Models Report and Summary:
print('============================================================')
print("Normal data distribution    : ", Counter(y_train))
print()
encoded = ce.OrdinalEncoder().fit_transform(X_train)
imputed = SimpleImputer(strategy = 'mean').fit_transform(encoded)
X_smote, y_smote = SMOTE().fit_sample(imputed, y_train)
print("SMOTE data distribution     : ", Counter(y_smote))
print()
X_nm, y_nm = NearMiss().fit_sample(imputed, y_train)
print("NearMiss data distribution  : ", Counter(y_nm))
print('============================================================')

Normal data distribution    :  Counter({1.0: 328, 0.0: 14})

SMOTE data distribution     :  Counter({0.0: 328, 1.0: 328})

NearMiss data distribution  :  Counter({0.0: 14, 1.0: 14})


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [119]:
# Classification report:
print("Normal model: \n", classification_report(y_val, prediction))
print('=============================================================================')
print("SMOTE Model: \n", classification_report_imbalanced(y_val, smote_prediction))
print('=============================================================================')
print("NearMiss Model: \n", classification_report_imbalanced(y_val, nm_prediction))

Normal model: 
               precision    recall  f1-score   support

         0.0       0.89      0.80      0.84        10
         1.0       0.97      0.99      0.98        76

    accuracy                           0.97        86
   macro avg       0.93      0.89      0.91        86
weighted avg       0.96      0.97      0.96        86

SMOTE Model: 
                    pre       rec       spe        f1       geo       iba       sup

        0.0       0.90      0.90      0.99      0.90      0.94      0.88        10
        1.0       0.99      0.99      0.90      0.99      0.94      0.90        76

avg / total       0.98      0.98      0.91      0.98      0.94      0.89        86

NearMiss Model: 
                    pre       rec       spe        f1       geo       iba       sup

        0.0       0.60      0.90      0.92      0.72      0.91      0.83        10
        1.0       0.99      0.92      0.90      0.95      0.91      0.83        76

avg / total       0.94      0.92      

In [120]:
# Model score:
print('Normal Pipeline Score: ', pipeline.score(X_val, y_val))
print('SMOTE Pipeline Score: ', smote_pipeline.score(X_val, y_val))
print('NearMiss Pipeline Score: ', nm_pipeline.score(X_val, y_val))

Normal Pipeline Score:  0.9651162790697675
SMOTE Pipeline Score:  0.9767441860465116
NearMiss Pipeline Score:  0.9186046511627907


In [121]:
# Apply the models to the test set:
test_prediction = model.predict(X_test)
test_smote_prediction = smote_model.predict(X_test)
test_nm_prediction = nm_model.predict(X_test)

In [122]:
# Models predictions:
test_prediction = pd.DataFrame(test_prediction, columns=['good_site'])
test_smote_prediction = pd.DataFrame(test_smote_prediction, columns=['good_site'])
test_nm_prediction = pd.DataFrame(test_nm_prediction, columns=['good_site'])

print("Normal Model:\n",test_prediction.value_counts())
print()
print("SMOTE Model:\n",test_smote_prediction.value_counts())
print()
print("NearMiss Model:\n",test_nm_prediction.value_counts())

Normal Model:
 good_site
0.0          1076
1.0           307
dtype: int64

SMOTE Model:
 good_site
0.0          1255
1.0           128
dtype: int64

NearMiss Model:
 good_site
0.0          1284
1.0            99
dtype: int64
